In [1]:
pip install dask[dataframe]

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [1]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
print("Vis setup Complete")
print("Setup complete.")

Vis setup Complete
Setup complete.


In [2]:
#Load the data using pandas read_csv function. This will generate a data frame we will use through the work.
cleaned_goalkeeeprs_df = pd.read_csv('goalkeepers_rating.csv')
cleaned_midfielders_df = pd.read_csv('midfielders_rating.csv')
cleaned_forwarders_df = pd.read_csv('forwarders_rating.csv')
cleaned_defenders_df = pd.read_csv('defenders_rating.csv')

In [3]:
#Show the first and last few records to check data is loading correctly
print(cleaned_defenders_df.head())
print(cleaned_defenders_df.tail())

         date pos_role  original_rating  goals  assists  shots_ontarget  \
0  2016-06-10       DC             6.56      0        0               0   
1  2016-06-10       DL             6.38      0        0               0   
2  2016-06-10       DC             6.80      0        0               0   
3  2016-06-10       DC             6.81      0        0               0   
4  2016-06-10       DR             6.59      0        0               0   

   shots_offtarget  shotsblocked  chances2score  drib_success  ...  \
0                0             0              0             0  ...   
1                1             1              0             0  ...   
2                0             0              0             0  ...   
3                0             0              0             0  ...   
4                0             0              0             0  ...   

   missed_penalties  owngoals  degree_centrality  betweenness_centrality  \
0                 0         0           1.071429    

In [4]:
import pandas as pd

df = cleaned_defenders_df

# Iterate through each column and check for columns with a single unique value
single_value_columns = [column for column in df.columns if df[column].nunique() == 1]

# Print columns with a single unique value
print("Columns with a single unique value:")
for column in single_value_columns:
    print(column)

Columns with a single unique value:
goals_ag_otb
goals_ag_itb
saves_itb
saves_otb
saved_pen


In [5]:
#we dropped the columns, that has single value in the dataset and it is known that all goal leepers here played for 90 minutes as well

columns_to_drop = ['goals_ag_otb',
'goals_ag_itb',
'saves_itb',
'saves_otb',
'saved_pen']

# Dropping the columns and retaining the name goalkeepers_df
cleaned_defenders_df = cleaned_defenders_df.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned dataframe to verify
cleaned_defenders_df.head()

,date,pos_role,original_rating,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,drib_success,...,missed_penalties,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,minutesPlayed,game_duration
0,2016-06-10,DC,6.56,0,0,0,0,0,0,0,...,0,0,1.071429,0.143055,0.603571,0.304348,0.000000,0.0,90,90
1,2016-06-10,DL,6.38,0,0,0,1,1,0,0,...,0,0,1.500000,0.321679,0.754464,0.478261,0.000000,0.0,90,90
2,2016-06-10,DC,6.80,0,0,0,0,0,0,0,...,0,0,1.071429,0.124054,0.574830,0.315068,2.917816,0.0,90,90
3,2016-06-10,DC,6.81,0,0,0,0,0,0,0,...,0,0,1.071429,0.200383,0.635338,0.275362,0.606906,0.0,90,90
4,2016-06-10,DR,6.59,0,0,0,0,0,0,0,...,0,0,1.000000,0.174008,0.603571,0.260870,0.553696,0.0,90,90


In [6]:
cleaned_defenders_df.shape

(5898, 48)

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import time

In [8]:
cleaned_defenders_df.head()

,date,pos_role,original_rating,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,drib_success,...,missed_penalties,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,minutesPlayed,game_duration
0,2016-06-10,DC,6.56,0,0,0,0,0,0,0,...,0,0,1.071429,0.143055,0.603571,0.304348,0.000000,0.0,90,90
1,2016-06-10,DL,6.38,0,0,0,1,1,0,0,...,0,0,1.500000,0.321679,0.754464,0.478261,0.000000,0.0,90,90
2,2016-06-10,DC,6.80,0,0,0,0,0,0,0,...,0,0,1.071429,0.124054,0.574830,0.315068,2.917816,0.0,90,90
3,2016-06-10,DC,6.81,0,0,0,0,0,0,0,...,0,0,1.071429,0.200383,0.635338,0.275362,0.606906,0.0,90,90
4,2016-06-10,DR,6.59,0,0,0,0,0,0,0,...,0,0,1.000000,0.174008,0.603571,0.260870,0.553696,0.0,90,90


In [9]:
cleaned_defenders_df.describe()

,original_rating,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,drib_success,drib_unsuccess,keypasses,...,missed_penalties,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,minutesPlayed,game_duration
count,5898.000000,5898.000000,5898.000000,5898.000000,5898.000000,5898.000000,5898.000000,5898.000000,5898.000000,5898.000000,...,5898.000000,5898.000000,5898.000000,5898.000000,5898.000000,5898.000000,5898.000000,5898.000000,5898.000000,5898.000000
mean,6.891948,0.036114,0.036114,0.120549,0.301797,0.094100,0.279078,0.348423,0.179044,0.159376,...,0.001017,0.007799,1.231311,0.191456,0.670072,0.392747,1.000193,0.005288,87.985758,90.300102
std,0.640150,0.187496,0.194597,0.355029,0.576635,0.321301,0.569509,0.704041,0.476856,0.453013,...,0.031882,0.087976,0.232959,0.069356,0.081990,0.113253,0.713930,0.017235,9.528790,2.985716
min,4.340000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.066667,0.000000,0.000000,0.014706,0.000000,0.000000,10.000000,90.000000
25%,6.460000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.071429,0.142812,0.622222,0.317460,0.474145,0.000000,90.000000,90.000000
50%,6.880000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.250000,0.185157,0.670635,0.393443,0.919554,0.000000,90.000000,90.000000
75%,7.290000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,1.400000,0.233941,0.725926,0.469388,1.445013,0.000000,90.000000,90.000000
max,10.000000,2.000000,3.000000,3.000000,5.000000,4.000000,4.000000,7.000000,4.000000,5.000000,...,1.000000,1.000000,1.857143,0.511700,0.933333,0.798077,4.551792,0.200000,120.000000,120.000000


In [10]:
cleaned_defenders_df.original_rating.value_counts()

original_rating
6.85    50
6.97    49
6.82    48
6.88    47
7.02    46
        ..
8.49     1
8.87     1
9.05     1
4.99     1
8.91     1
Name: count, Length: 389, dtype: int64

In [11]:
# lets move towards splitting the data.

# get the locations
y = cleaned_defenders_df['original_rating']
X = cleaned_defenders_df.drop(columns=['original_rating'], axis=1)

In [12]:
X.head()

,date,pos_role,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,drib_success,drib_unsuccess,...,missed_penalties,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,minutesPlayed,game_duration
0,2016-06-10,DC,0,0,0,0,0,0,0,0,...,0,0,1.071429,0.143055,0.603571,0.304348,0.000000,0.0,90,90
1,2016-06-10,DL,0,0,0,1,1,0,0,0,...,0,0,1.500000,0.321679,0.754464,0.478261,0.000000,0.0,90,90
2,2016-06-10,DC,0,0,0,0,0,0,0,0,...,0,0,1.071429,0.124054,0.574830,0.315068,2.917816,0.0,90,90
3,2016-06-10,DC,0,0,0,0,0,0,0,0,...,0,0,1.071429,0.200383,0.635338,0.275362,0.606906,0.0,90,90
4,2016-06-10,DR,0,0,0,0,0,0,0,1,...,0,0,1.000000,0.174008,0.603571,0.260870,0.553696,0.0,90,90


In [13]:
y.head()

0    6.56
1    6.38
2    6.80
3    6.81
4    6.59
Name: original_rating, dtype: float64

In [14]:
# Create DataFrame
# Convert 'date' column to datetime with error handling
X['date'] = pd.to_datetime(X['date'], errors='coerce')

# Check for any NaT values
if X['date'].isna().any():
    print("Some dates couldn't be parsed. Handling them as NaT (Not a Time).")

# Extract year, month, and day
X['year'] = X['date'].dt.year
X['month'] = X['date'].dt.month
X['day'] = X['date'].dt.day

# Drop the original 'date' column
X = X.drop(columns=['date'])

# Display the DataFrame
print(X.head())

  pos_role  goals  assists  shots_ontarget  shots_offtarget  shotsblocked  \
0       DC      0        0               0                0             0   
1       DL      0        0               0                1             1   
2       DC      0        0               0                0             0   
3       DC      0        0               0                0             0   
4       DR      0        0               0                0             0   

   chances2score  drib_success  drib_unsuccess  keypasses  ...  \
0              0             0               0          0  ...   
1              0             0               0          0  ...   
2              0             0               0          0  ...   
3              0             0               0          0  ...   
4              0             0               1          0  ...   

   betweenness_centrality  closeness_centrality  flow_centrality  \
0                0.143055              0.603571         0.304348   
1   

In [15]:
X.columns

Index(['pos_role', 'goals', 'assists', 'shots_ontarget', 'shots_offtarget',
       'shotsblocked', 'chances2score', 'drib_success', 'drib_unsuccess',
       'keypasses', 'touches', 'passes_acc', 'passes_inacc', 'crosses_acc',
       'crosses_inacc', 'lballs_acc', 'lballs_inacc', 'grduels_w', 'grduels_l',
       'aerials_w', 'aerials_l', 'poss_lost', 'fouls', 'wasfouled',
       'clearances', 'stop_shots', 'interceptions', 'tackles', 'dribbled_past',
       'tballs_acc', 'tballs_inacc', 'ycards', 'rcards', 'dangmistakes',
       'countattack', 'offsides', 'missed_penalties', 'owngoals',
       'degree_centrality', 'betweenness_centrality', 'closeness_centrality',
       'flow_centrality', 'flow_success', 'betweenness2goals', 'minutesPlayed',
       'game_duration', 'year', 'month', 'day'],
      dtype='object')

In [16]:
X.dtypes

pos_role                   object
goals                       int64
assists                     int64
shots_ontarget              int64
shots_offtarget             int64
shotsblocked                int64
chances2score               int64
drib_success                int64
drib_unsuccess              int64
keypasses                   int64
touches                     int64
passes_acc                  int64
passes_inacc                int64
crosses_acc                 int64
crosses_inacc               int64
lballs_acc                  int64
lballs_inacc                int64
grduels_w                   int64
grduels_l                   int64
aerials_w                   int64
aerials_l                   int64
poss_lost                   int64
fouls                       int64
wasfouled                   int64
clearances                  int64
stop_shots                  int64
interceptions               int64
tackles                     int64
dribbled_past               int64
tballs_acc    

In [17]:
X.pos_role.value_counts()

pos_role
DC    3604
DL    1147
DR    1147
Name: count, dtype: int64

In [18]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

numeric_transformer = MinMaxScaler()  
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)
X = preprocessor.fit_transform(X)
# Get the numerical column names after scaling
num_col_names = num_features

# Get the categorical column names after one hot encoding
cat_col_names = preprocessor.named_transformers_['OneHotEncoder'].get_feature_names_out(cat_features)

# Combine all column names
all_col_names = list(num_col_names) + list(cat_col_names)
X= pd.DataFrame(X, columns=all_col_names)
print(X.head())

   goals  assists  shots_ontarget  shots_offtarget  shotsblocked  \
0    1.0      0.0             0.0              0.0           0.0   
1    0.0      1.0             0.0              0.0           0.0   
2    1.0      0.0             0.0              0.0           0.0   
3    1.0      0.0             0.0              0.0           0.0   
4    0.0      0.0             1.0              0.0           0.0   

   chances2score  drib_success  drib_unsuccess  keypasses  touches  ...  \
0            0.0           0.0            0.00        0.0      0.0  ...   
1            0.0           0.2            0.25        0.0      0.0  ...   
2            0.0           0.0            0.00        0.0      0.0  ...   
3            0.0           0.0            0.00        0.0      0.0  ...   
4            0.0           0.0            0.00        0.0      0.0  ...   

   flow_success  betweenness2goals  minutesPlayed  game_duration  year  \
0      0.279569           0.646684       0.369738       0.000000  

In [19]:
X.shape

(5898, 51)

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now, split the train+validation set into separate training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Display the sizes of the splits to ensure they are correct
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 3538
Validation set size: 1180
Test set size: 1180


In [21]:
X_train.head()

,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,drib_success,drib_unsuccess,keypasses,touches,...,flow_success,betweenness2goals,minutesPlayed,game_duration,year,month,day,pos_role_DC,pos_role_DL,pos_role_DR
947,1.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.25,0.0,...,0.561709,0.808355,0.558707,0.296296,0.0,0.727273,0.0,0.5,0.727273,0.500000
249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.381893,0.760804,0.697650,0.273224,0.0,0.727273,0.0,0.0,0.454545,0.700000
5514,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.491270,0.760804,0.445422,0.076154,0.0,0.727273,0.0,1.0,0.454545,0.566667
3232,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.259519,0.646684,0.362530,0.229885,0.0,0.727273,0.0,1.0,0.000000,0.433333
824,1.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.25,0.0,...,0.344510,0.718537,0.479387,0.085470,0.0,0.727273,0.0,0.5,0.727273,0.266667


In [57]:
# Save each DataFrame as a CSV file
X.to_csv('defenders_rating_X.csv', index=False)
y.to_csv('defenders_rating_y.csv', index=False)


print("DataFrames have been saved as CSV files.")

DataFrames have been saved as CSV files.
